# Auth Weekly Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
import psycopg2
import pandas as pd
import matplotlib
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from IPython.core.display import HTML

%load_ext sql
%config SqlMagic.displaylimit = 5

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
connect_to_db = 'postgresql://' + \
                os.getenv('PG_USER', '') + ":" + os.getenv('PG_PASSWORD', '') +'@' + \
                os.getenv('PG_HOST', '') + ':' + os.getenv('PG_PORT', '5432') + '/' + os.getenv('PG_DB_NAME', '');
    
%sql $connect_to_db

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql 
select now() AT TIME ZONE 'PST' as current_date

weekly total before running time.

In [ ]:
%%sql stat_accounts_weekly_completed  <<
SELECT count(*) FILTER (WHERE type_code = 'BASIC') AS BASIC
,count(*) FILTER (WHERE type_code = 'PREMIUM') AS PREMIUM
FROM orgs WHERE status_code='ACTIVE'
AND date(created at time zone 'utc' at time zone 'pst') > date(current_date - 1 - interval '1 weeks')
AND date(created at time zone 'utc' at time zone 'pst') <= date(current_date - 1)

Save to CSV

In [ ]:
filename = os.getenv('DATA_DIR', '')+'auth_weekly_stats_till_' + datetime.strftime(datetime.now()-timedelta(1), '%Y-%m-%d') +'.csv'

with open(filename, 'w') as f:      
    f.write('Number of Accounts:\n')
    stat_accounts_weekly_completed.DataFrame().to_csv(f, sep=',', encoding='utf-8', index=False)

In [ ]:
%%sql stat_users_weekly_completed  <<
SELECT count(*) FILTER (WHERE login_source = 'BCEID') AS BCEID
,count(*) FILTER (WHERE login_source = 'BCSC') AS BCSC
FROM users
WHERE date(login_time at time zone 'utc' at time zone 'pst') > date(current_date - 1 - interval '1 weeks')
AND date(login_time at time zone 'utc' at time zone 'pst') <= date(current_date - 1)

In [ ]:
with open(filename, 'a') as f:      
    f.write('\n\n Number of Login Users:\n')
    stat_users_weekly_completed.DataFrame().to_csv(f, sep=',', encoding='utf-8', index=False)